In [ ]:
import os
import pathlib

# URL del repository pubblico
REPO_URL = 'https://github.com/ev91/MachineInnovators_Inc_ProAI.git'
repo_dir = pathlib.Path('/content/MachineInnovators_Inc_ProAI')

if not repo_dir.exists():
    print(f"[*] Clonazione di {REPO_URL}...")
    !git clone $REPO_URL
    print(f"[✓] Repository clonato in {repo_dir}")
else:
    print(f"[*] Repository già presente in {repo_dir}")

os.chdir(repo_dir)
print(f"[✓] Working directory: {repo_dir}")

## 2) Installa le dipendenze

In [ ]:
import sys
import os
import pathlib

project_root = pathlib.Path.cwd()

# Verifica che il repo sia nella working directory
if not (project_root / 'src').exists():
    alt = pathlib.Path('/content/MachineInnovators_Inc_ProAI')
    if alt.exists():
        project_root = alt
        os.chdir(project_root)
    else:
        raise RuntimeError('Repository non trovato: esegui la cella precedente per clonare')

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"[*] Installazione dipendenze da {project_root / 'requirements.txt'}...")
!pip install -q -r requirements.txt
print(f"[✓] Dipendenze installate")

# Configurazione MLflow
os.environ['MLFLOW_TRACKING_URI'] = f'file://{project_root / "mlruns"}'
os.environ['REGISTERED_MODEL_NAME'] = 'Sentiment'

print(f"[✓] MLflow tracking URI: {os.environ['MLFLOW_TRACKING_URI']}")
print(f"[✓] Model name: {os.environ['REGISTERED_MODEL_NAME']}")

## 3) Demo: Inferenza con il modello HuggingFace

Usiamo il modello pre-addestrato `cardiffnlp/twitter-roberta-base-sentiment-latest` per fare una predizione rapida.

In [ ]:
from src.serving.load_model import predict_fn

# Test su diversi testi
test_texts = [
    "I absolutely love this product!",
    "This is just okay, nothing special.",
    "Terrible experience, would not recommend.",
]

print("[*] Predizioni usando il modello HuggingFace:")
print("-" * 60)

for text in test_texts:
    label, score = predict_fn(text)
    print(f"Text: {text[:50]:50s} | Label: {label:8s} | Score: {score:.4f}")

print("[✓] Inferenza completata")

## 4) Addestra una nuova versione del modello

Questo step registra una nuova versione del modello nel MLflow Model Registry (file-based).

In [ ]:
print("[*] Avvio training...")
!python -m src.models.train_roberta --experiment sentiment_colab
print("[✓] Training completato")

## 5) Recupera l'ultima versione registrata dal Model Registry

In [ ]:
from mlflow.tracking import MlflowClient

model_name = os.environ.get('REGISTERED_MODEL_NAME', 'Sentiment')
tracking_uri = os.environ['MLFLOW_TRACKING_URI']

client = MlflowClient(tracking_uri=tracking_uri)

# Recupera tutte le versioni
all_versions = client.search_model_versions(f"name='{model_name}'")

if not all_versions:
    raise RuntimeError(f"Nessuna versione trovata per il modello '{model_name}'")

# Prendi la versione più recente
latest = sorted(all_versions, key=lambda v: int(v.version))[-1]
new_model_uri = f"models:/{model_name}/{latest.version}"

print(f"[✓] Ultime versioni registrate:")
for v in sorted(all_versions[-3:], key=lambda x: int(x.version), reverse=True):
    print(f"  - Version {v.version}: stage={v.current_stage}")

print(f"[✓] Usando versione: {new_model_uri}")

## 6) Valuta e promuovi a Production

Usiamo `data/holdout.csv` come set di valutazione. Se il nuovo modello è >= della versione in Production (o se nessuno è in Production), viene promosso.

In [ ]:
print("[*] Valutazione del modello e promozione a Production...")
!python -m src.models.evaluate --new_model_uri $new_model_uri --eval_csv data/holdout.csv --min_improvement 0.0
print("[✓] Valutazione e promozione completate")

## 7) Carica il modello in Production dal registry

In [ ]:
from src.utils.mlflow_utils import get_production_model_uri

prod_uri = get_production_model_uri(model_name)

if prod_uri:
    os.environ['MODEL_URI'] = prod_uri
    print(f"[✓] Production URI trovata: {prod_uri}")
else:
    os.environ['MODEL_URI'] = ''
    print(f"[!] Production URI non trovata, fallback a HuggingFace")

## 8) Inferenza usando il modello in Production

Ora usiamo il modello registrato in Production (se disponibile) altrimenti fallback su HuggingFace.

In [ ]:
# Reload predict_fn per utilizzare il nuovo MODEL_URI
import importlib
import src.serving.load_model
importlib.reload(src.serving.load_model)
from src.serving.load_model import predict_fn

print(f"[*] Predizioni usando MODEL_URI={os.environ.get('MODEL_URI', 'None (fallback HF)')}")
print("-" * 60)

test_texts = [
    "This is amazing! Best service ever!",
    "Not the best, could be better.",
    "Absolutely horrible, never again!",
]

for text in test_texts:
    label, score = predict_fn(text)
    print(f"Text: {text[:50]:50s} | Label: {label:8s} | Score: {score:.4f}")

print("[✓] Inferenza completata")

## 9) Sistema di Monitoraggio Continuo

Questo progetto implementa un **sistema completo di monitoring** con Prometheus e Grafana per valutare continuamente:

- **Performance dell'API**: volume di richieste, latenza (p50/p90), tasso d'errore
- **Sentiment Analysis**: distribuzione dei sentiment predetti (positive/neutral/negative) nel tempo
- **Performance del Modello**: F1-score e accuracy monitorati ad ogni retraining
- **Data Drift Detection**: flag binario e timeline dello shift dei dati rispetto alla baseline

### Flusso di Dati di Monitoraggio

```
┌─────────────────┐
│  FastAPI /predict│─────── Espone metriche Prometheus ──────┐
└─────────────────┘                                          │
                                                             │
                                                    ┌────────▼──────────┐
┌────────────────────────┐                        │   Prometheus      │
│ Airflow DAG            │─ Pushes model metrics ─│   (Data Store)    │
│ (evaluate_and_promote) │                        └────────┬──────────┘
└────────────────────────┘                                 │
                                                           │
                                                    ┌──────▼────────┐
                                                    │    Grafana    │
                                                    │   Dashboard   │
                                                    └───────────────┘
```

### Come Leggerlo in Tempo Reale

Una volta avviato lo stack con `docker compose up`:

1. **Apri Grafana**: http://localhost:3000 (login: admin/admin)
2. **Seleziona il dashboard**: "MLOps – Sentiment Analysis Monitoring"
3. **Pannelli principali**:
   - 📊 **API Request Rate**: volume di richieste al `/predict` endpoint (req/s)
   - ⚠️ **Error Rate**: percentuale di errori API nel tempo
   - ⏱️ **Latency (p50/p90)**: mediana e 90° percentile della latenza
   - 🚨 **Data Drift Status**: segnale rosso quando drift viene rilevato
   - 💭 **Sentiment Distribution**: pie chart della distribuzione positive/neutral/negative (5m avg)
   - 📈 **Sentiment Predictions**: andamento delle predizioni per label nel tempo
   - 🎯 **Model Performance**: F1-score e accuracy tracciati ad ogni valutazione
   - 📊 **Latest F1 Score**: indicatore del valore attuale di F1
   - 🔄 **Predictions Distribution**: distribuzione delle predizioni nell'ultima ora
   - 📉 **Data Drift Timeline**: cronologia del flag di drift nel tempo

## 10) Prossimi passi

### Per eseguire lo stack completo (Airflow + Prometheus + Grafana + Monitoraggio)
1. Clona il repository localmente
2. Assicurati di avere Docker e Docker Compose installati (v2.0+)
3. Esegui:
   ```bash
   git clone https://github.com/ev91/MachineInnovators_Inc_ProAI.git
   cd MachineInnovators_Inc_ProAI
   docker compose up --build
   ```
4. Attendi ~60-90 secondi finché tutti i servizi sono in running:
   - **FastAPI**: http://localhost:8000 → Inference endpoint
   - **MLflow UI**: http://localhost:5000 → Model Registry
   - **Airflow UI**: http://localhost:8080 → DAG orchestration
   - **Grafana**: http://localhost:3000 (admin/admin) → **Monitoring Dashboard**
   - **Prometheus**: http://localhost:9090 → Metrics database

### Verifiche rapide

**Health check API:**
```bash
curl http://localhost:8000/health
```

**Predizione di sentiment:**
```bash
curl -X POST http://localhost:8000/predict \
  -H 'Content-Type: application/json' \
  -d '{"text": "I love this product!"}'
```

**Metriche Prometheus disponibili:**
```bash
curl http://localhost:8000/metrics
```

### Per simulare data drift e innescare il retraining automatico
Vedi [docs/data_drift_simulation.md](https://github.com/ev91/MachineInnovators_Inc_ProAI/blob/main/docs/data_drift_simulation.md)

### Per comprendere il flusso di training, valutazione e promozione
Vedi [docs/training_and_promotion.md](https://github.com/ev91/MachineInnovators_Inc_ProAI/blob/main/docs/training_and_promotion.md)

### Documentazione completa
Leggi il [README.md](https://github.com/ev91/MachineInnovators_Inc_ProAI) per la panoramica architetturale completa.

## 11) Screenshot (evidenze)

Le seguenti schermate mostrano lo stack in esecuzione. Una volta avviato con `docker compose up`, potrai accedere a questi servizi e visualizzare il monitoraggio in tempo reale del modello di sentiment analysis.

![Airflow DAG](../docs/screenshots/Schermata%20Airflow.png)

**Airflow DAG Orchestration** (`http://localhost:8080`): Vista del DAG `retrain_sentiment` con tutti i task (ingest → drift → branch → train → evaluate_and_promote). Mostra i run completati, lo stato di ogni task e i log per verificare il flusso di retraining automatico quando viene rilevato data drift.

![Grafana Dashboard](../docs/screenshots/Schermata%20Grafana.png)

**Grafana Dashboard di Monitoraggio** (`http://localhost:3000`, login: admin/admin): Dashboard "MLOps – Sentiment Analysis Monitoring" che fornisce **visibilità continua** su:
- **Request Rate & Error Rate**: volume di richieste API e percentuale d'errore
- **Latency (p50/p90)**: performance dell'endpoint di inferenza
- **Sentiment Distribution**: pie chart della proporzione positive/neutral/negative nei sentiment predetti (ultimi 5 minuti)
- **Sentiment Predictions**: trend temporale di quanti testi sono stati classificati come positive/neutral/negative
- **Model Performance (F1 & Accuracy)**: metriche di performance del modello nel tempo, aggiornate dopo ogni retraining
- **Data Drift Timeline**: cronologia del flag di drift (0=no drift, 1=drift detected)

Questa è la **componente chiave** per la "continuous monitoring" richiesta dalla traccia d'esame.

![MLflow Registry](../docs/screenshots/Schermata%20MLFlow.png)

**MLflow Model Registry** (`http://localhost:5000`): Model Registry con tutte le versioni del modello `Sentiment`. Mostra:
- Versioni registrate durante il training
- Stage corrente di ogni versione (Production/Staging/None)
- Parametri e metriche di ogni run (accuracy, F1-score, ecc.)
- Automaticamente aggiornato dal DAG di Airflow durante valutazione e promozione

![GitHub Actions CI](../docs/screenshots/Schermata%20Github%20Actions.png)

**GitHub Actions CI/CD Pipeline**: Automaticamente eseguita su ogni push/PR con:
- Linting (ruff, flake8)
- Test unitari (pytest)
- Smoke test su training e inferenza
- Utile come prova che la pipeline è funzionante e mantenuta